# PCA Lab

In this lab we will be practicing using PCA to analyze a genomic dataset


## The data

*Arabidopsis thaliana*. This data was downloaded from [The 1001 Genomes Project](https://1001genomes.org/data/GMI-MPI/releases/v3.1) and the file is called `1001genomes_snp-short-indel_only_ACGTN.vcf.gz` and is in the '/mnt/research/PLB812_FS24_S001/12_PCA' folder.

## Running PCA 

We will be using the software Plink to run PCA. Our first step will be loading the Plink module on the hpcc.

`$module purge` <br>
`$module load PLINK/2.00a3.7-gfbf-2023a`

Next, if your data is in a vcf file, you need to convert it to a plink format.  We will also want to filter these files to give us SNPs that are not in linkage disequilibrium with each other, and to remove rare sites or sites with a lot of missing data. You can look up each of the options in the following command to see how they work.

`$plink2 -bfile [file-prefix] --indep-pairwise 100 20 0.2 --geno 0.05 --max-alleles 2 --maf 0.05 --allow-extra-chr --make-bed --out [file-prefix]_filtered`

If you were to do this another time and your data wass already in a plink format (file names end with **.bed** and **.bim** and **.fam** and have the same file prefix), you could run the following command:

`$plink2 -bfile [file-prefix] --indep-pairwise 100 20 0.2 --geno 0.05 --max-alleles 2 --maf 0.05 --allow-extra-chr --make-bed --out [file-prefix].filtered`

Once you have filtered your plink files of SNPs, it is time to run the PCA analysis!

`$plink2 -bfile [file-prefix] --extract [file-prefix].filtered.in --pca --allow-extra-chr --out [file-prefix]-pca`

Note the `--extract` flag here directs you towards a list of sites that was generated in the previous filtered set. If you were to skip this, plink would try to run the PCA on all the sites, not just the sites that passed the filters. 

You can use **less** to look at the output files from this step, which will be `[file-prefix]-pca.eigenvec` and `[file-prefix]-pca.eigenval`.

## Subsampling SNPs

We talked in class about how the number of SNPs is important for making a PCA. In the lab we'll investigate this using the *A. thaliana* data.

We can efficiently subsample a smaller dataset by sampling from the [file-prefix].filtered.in site, and then only using these sites when we run the PCA. To subsample 100 sites, use the following bash command:

`shuf [file-prefix].filtered.in | head -n 100 > [file-prefix].100.in`

Take a second to look at this command and think about what the different parts of it do (or ask a neighbor or Emily!).

Modify the command to take subsamples of 100, 1000, and 10,000 sites.

You can count the number of lines in your `[file-prefix].x.in` files using `wc -l [file]`. Use this to check that your subsampling worked correctly. Then, rerun your PCA on these subsamples.

In [ ]:
rosseli5@dev-amd20:~/08_PCA$ cd /mnt/research/PLB812_FS24_S001/12_PCA
rosseli5@dev-amd20:/mnt/research/PLB812_FS24_S001/12_PCA$ ls
1001genomes_snp-short-indel_only_ACGTN.vcf.gz  arabidopsis.bim
accessions.csv                                 arabidopsis.fam
arabidopsis.bed                                pca-plots.Rmd
#moving back home
rosseli5@dev-amd20:/mnt/research/PLB812_FS24_S001/12_PCA$ cd /mnt/home/rosse
li5/08_PCA
#converting our vcf file into plink's
#this takes a while, Emily says we can skip ahead to the next step but we're just going to chill because I have a fear of breaking terminal by overloading it with stuff
#if I have to finish this on Friday, that's fine
rosseli5@dev-amd20:~/08_PCA$plink2 -vcf /mnt/research/PLB812_FS24_S001/12_PCA/1001genomes_snp-short-indel_only_ACGTN.vcf.gz --indep-pairwise 100 20 0.2 --geno 0.05 --max-alleles 2 --maf 0.05 --allow-extra-chr --make-bed --out /mnt/home/rosseli5/08_PCA/1001genomes_snp-short-indel_only_ACGTN_filtered
#checking to make sure files exist
rosseli5@dev-amd20:~/08_PCA$ ls
#yay, we have a bed, a bim, a fam, and a log
#success
#JK LOL NOT SUCCESS
#We were also supposed to get a file ending in .in, this is an error with Emily's code
#We're saying f-it, and using the files that she preparepared.
rosseli5@dev-amd20:~/08_PCA$ cd /mnt/research/PLB812_FS24_S001/12_PCA
#Emily's files are called arabidopsis
rosseli5@dev-amd20:~/08_PCA$ plink2 -bfile /mnt/research/PLB812_FS24_S001/12_PCA/arabidopsis --extract /mnt/research/PLB812_FS24_S001/12_PCA/arabidopsis.prune.in --pca --allow-extra-chr --out /mnt/home/rosseli5/08_PCA/arabidopsis-pca
#ran into an error code
#libgomp: Thread creation failed: Resource temporarily unavailable
#moving nodes
rosseli5@dev-amd20:~/08_PCA$ ssh dev-intel18
rosseli5@dev-intel18:~$ cd /mnt/home/rosseli5/08_PCA
#had to reload plink
rosseli5@dev-intel18:~/08_PCA$ module load PLINK/2.00a3.7-gfbf-2023a
#this will take a while to run
rosseli5@dev-intel18:~/08_PCA$ plink2 -bfile /mnt/research/PLB812_FS24_S001/
12_PCA/arabidopsis --extract /mnt/research/PLB812_FS24_S001/12_PCA/arabidops
is.prune.in --pca --allow-extra-chr --out /mnt/home/rosseli5/08_PCA/arabidop
sis-pca
#subsetting
rosseli5@dev-intel18:~/08_PCA$ shuf /mnt/research/PLB812_FS24_S001/12_PCA/arabidopsis.prune.in | head -n 100 > /mnt/home/rosseli5/08_PCA/arabidopsis.100.in
rosseli5@dev-intel18:~/08_PCA$ shuf /mnt/research/PLB812_FS24_S001/12_PCA/arabidopsis.prune.in | head -n 1000 > /mnt/home/rosseli5/08_PCA/arabidopsis.1000.in
rosseli5@dev-intel18:~/08_PCA$ shuf /mnt/research/PLB812_FS24_S001/12_PCA/arabidopsis.prune.in | head -n 10000 > /mnt/home/rosseli5/08_PCA/arabidopsis.10000.in
#Rerun the data through the filtered.in files
rosseli5@dev-amd20:~/08_PCA$ plink2 -bfile /mnt/research/PLB812_FS24_S001/12_PCA/arabidopsis --extract /m
nt/home/rosseli5/08_PCA/arabidopsis.100.in --pca --allow-extra-chr --out /mnt/home/rosseli5/08_PCA/arabid
opsis100-pca
rosseli5@dev-amd20:~/08_PCA$ plink2 -bfile /mnt/research/PLB812_FS24_S001/12_PCA/arabidopsis --extract /mnt/ho
me/rosseli5/08_PCA/arabidopsis.1000.in --pca --allow-extra-chr --out /mnt/home/rosseli5/08_PCA/arabidopsi
s1000-pca
rosseli5@dev-intel18:~$ plink2 -bfile /mnt/research/PLB812_FS24_S001/12_PCA/arabidopsis --extract /mnt/ho
me/rosseli5/08_PCA/arabidopsis.10000.in --pca --allow-extra-chr --out /mnt/home/rosseli5/08_PCA/arabidops
is10000-pca
#Check if subsampleing worked
rosseli5@dev-intel18:~/08_PCA$ wc -l arabidopsis.100.in
#getting the answer for homework question 2
rosseli5@dev-intel18:~/08_PCA$ wc -l /mnt/research/PLB812_FS24_S001/12_PCA/arabidopsis.prune.in

## Making plots

You will need some extra information about the genotypes in your PCA.

This file is called `accessions.csv`.

There is a file called 'pca-plots.Rmd' in the class folder ('/mnt/research/PLB812_FS24_S001/12_PCA') that has code for making PCA plots. Please open it up in Rstudio, edit the paths for your own data, and make PCA plots for all the SNPs and then for PCAs made from each subset.

When you're done, put the PCA plots in this notebook.

## Questions

1. Based on the plot you've made, what are the major axes of variation in your diversity dataset? How do you know this?

The most diversity from the sampled plants is coming from Germany and Asia. In the PC1 vs PC2 PCA plot, Germany is not grouping, and while some plants sampled from this region are clustering with the rest of Europe, a large portion of the sampled plants show large variation when compared to other German plants and when compared to other regions. Additionally, while plants sampled from Asia are clustering together, they are distinct from all other regions.
   
2. How many sites were in the dataset of all SNPs that you used to calculate the PCA without subsampling? How did you find this out?

Using the code wc -l /mnt/research/PLB812_FS24_S001/12_PCA/arabidopsis.prune.in, I found that 41060 SNPs were in the unfiltered data set

3. How did changing the number of SNPs affect the PCA plots you made? How many SNPs would you recommend that another researcher use for Arabidopsis PCA?

The clustering of regions became increasingly clear with the addition of SNPs; for example, 100 was less clear than 10000. I recommend that a researcher use either the 10000 SNPs or the unfiltered set. While the PCA plots look very similar for the unfiltered vs the 10000 SNP set, using the unfiltered set allows you to skip a programming step, but it does take a while to get the .eigenvec file. If you have the computational power and time, unfiltered, if not, 10000 is fine.

## Other helpful stuff:

[The Documentation for Plink2](https://www.cog-genomics.org/plink/2.0/)

[A tutorial on PCA using Plink](https://www.zoology.ubc.ca/~schluter/R/Genomics.html#PCA_with_PLINK2)